**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 2 - Open Science Platform - User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [10]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [11]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [12]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-01-30 17:51:52,211 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-01-30 17:51:52.212 | INFO     | __main__:<module>:3 - b'70bc2ddd69f6b74940402d8e757c10910c211263beb055f78fb1824da9596c21'


2 - Creates a user account into the blockchain.

In [19]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-01-30 20:36:44.196 | INFO     | __main__:<module>:13 - User Role: author
2025-01-30 20:36:44.197 | INFO     | __main__:<module>:24 - Creating account with name: angry_proskuriakova
2025-01-30 20:36:44.200 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-30 20:36:44.224 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'70bc2ddd69f6b74940402d8e757c10910c211263beb055f78fb1824da9596c21'
2025-01-30 20:36:44.226 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 7F527500292905F7FFDABA710453F1EAEFAC9CF7
2025-01-30 20:36:44.227 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-30 20:36:44.228 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-01-30 20:36:44,243 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-30 20:36:44,244 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 

b'499c91eaa0c11a244ea55b16c077eab04f10a49bb8c59a89b875dc5e1ac6ac07'

b'61c8d3a73ee1288db2bab69e6c4cd981a5e850a61a03dd25a2ea74db7b26469f'

3 - Queries the blockchain to confirm the succesful creation of the account.

In [20]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-01-30 20:40:09.157 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-30 20:40:09,162 - WARNING - create_account_contract_address: 7F527500292905F7FFDABA710453F1EAEFAC9CF7
2025-01-30 20:40:09,181 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-30 20:40:09,183 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-30 20:40:09,703 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-30 20:40:09,711 - INFO - ('COMMITTED', 5, 0)
2025-01-30 20:40:09,713 - WARNING - Account Data: {'address': '7F527500292905F7FFDABA710453F1EAEFAC9CF7', 'hex_hash': '1cbbace6ef6102522173d29b372165a8527fd56aa1c25b334898f8b5feef6bac'}
2025-01-30 20:40:09.714 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-30 20:40:09,718 - INFO - Appended data under 'angry_proskuriakova@test' in logs/account_data.json
2025-01-30 20:40:09.719 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-30 20:40:09.721 | DEBUG    | 

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [15]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-01-30 17:51:56,487 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-01-30 17:51:56.489 | INFO     | __main__:<module>:3 - b'be50112dce211058fe403dfe4cbfe7480241eb2ab7ae0748f3208ee87661befa'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [21]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-01-30 20:42:46.672 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-30 20:42:46.697 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'be50112dce211058fe403dfe4cbfe7480241eb2ab7ae0748f3208ee87661befa'
2025-01-30 20:42:46.698 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: D653338D77F4F894C5D46654F4119FB9DB64751D
2025-01-30 20:42:46.699 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-30 20:42:46.701 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-01-30 20:42:46,703 - INFO - Processing account: angry_proskuriakova@test
2025-01-30 20:42:46,707 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Angry Proskuriakova', 'foaf:mbox': 'angry_proskuriakova@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Illinois College of Opt

6 - Queries the blockchain again to confirm the succesfull details setting.

In [22]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid = process_raw_data(details)

2025-01-30 21:00:19.899 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-30 21:00:19,904 - WARNING - create_account_contract_address: 7F527500292905F7FFDABA710453F1EAEFAC9CF7
2025-01-30 21:00:19,932 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-30 21:00:19,935 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-30 21:00:20,247 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-30 21:00:20,258 - INFO - ('COMMITTED', 5, 0)
2025-01-30 21:00:20,261 - WARNING - Account Data: {'address': '7F527500292905F7FFDABA710453F1EAEFAC9CF7', 'hex_hash': 'b6095814385c9fcec5290646ae388fbb9deefba9f16fb6e915965c74abc1bf4f'}
2025-01-30 21:00:20.265 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-30 21:00:20,272 - INFO - Appended data under 'angry_proskuriakova@test' in logs/account_data.json
2025-01-30 21:00:20.274 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-30 21:00:20.277 | DEBUG    | 

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [18]:
user_metadata = download_json_from_ipfs(account_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-01-30 17:52:02.579 | INFO     | __main__:<module>:2 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'youthful_neumann@test', 'schema:publicKey': '7c90913b8b28fdc78e3cebc5adbb391b6afe81f98c01fb894252deed6ef93d56', 'schema:roleName': 'publisher'}, 'foaf:mbox': 'youthful_neumann@email.com', 'foaf:name': 'Youthful Neumann', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Allahabad University'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '7087-9949-7478-X'}}
